# Bell's States

This notebook is thought to play with Bell's States and entanglement using Qiskit tools. Let's load the modules we need. 

In [ ]:
# pacchetti di visualizzazione e numerici 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# importiamo il Qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, IBMQ, execute

# importiamo i tool di grafica del Qiskit
from qiskit.tools.visualization import matplotlib_circuit_drawer as circuit_drawer
from qiskit.tools.visualization import plot_histogram, qx_color_scheme, plot_state_city, plot_bloch_multivector, plot_state_paulivec, plot_state_hinton, plot_state_qsphere

## Measuring Bell's States
### Reduced version of the Notebook by:
Jay Gambetta, Antonio Córcoles, Andrew Cross, Anna Phan

We want to create Bell's States and measure them in different basis. To do this we will use the Aer qasm simulator.
**NB** if you exit from the session or if you need to update your API Token then uncomment the following line of code accordingly.


In [ ]:
backend = Aer.get_backend('qasm_simulator') # select the qasm simulator

# to use a real processor:
#IBMQ.save_account('')   
#IBMQ.load_accounts()
#from qiskit.providers.ibmq import least_busy
#backend = least_busy(IBMQ.backends(operational=True, simulator=False))
#print("the best backend is " + backend.name())

Working with the qiskit, the rule is to name the states writing the first qubit on the right. In this way, the tensor product of $q_0$ and $q_1$ is $q_1\otimes q_0$.

Using $q_0$ as control and $q_1$ as target, the CNOT is thus written:

$$ CNOT =\begin{pmatrix} 1 & 0 & 0 & 0\\ 0 & 0 & 0 & 1\\0& 0& 1 & 0\\0 & 1 & 0 & 0 \end{pmatrix},$$

One of the four Bell's States can be created by applying a gate $H$ on the control qubit, in our example $q_0$:

$$ |Bell \rangle= \frac{(|00 \rangle+|11\rangle)} {\sqrt{2}} $$


In [ ]:
# Create the registers
q2 = QuantumRegister(2)
c2 = ClassicalRegister(2)

# Create the Bell's State
bell = QuantumCircuit(q2, c2)
bell.h(q2[0])
bell.cx(q2[0], q2[1])

# Measure the qubits in the standard base (Z)
measureZZ = QuantumCircuit(q2, c2)
measureZZ.measure(q2[0], c2[0])
measureZZ.measure(q2[1], c2[1])
bellZZ = bell+measureZZ

# Measure the qubits in the superposition base (X)
measureXX = QuantumCircuit(q2, c2)
measureXX.h(q2)
measureXX.measure(q2[0], c2[0])
measureXX.measure(q2[1], c2[1])
bellXX = bell+measureXX

Let's draw the circuit that maesures in Z basis:


In [ ]:
circuit_drawer(bellZZ,style=qx_color_scheme())

How can we draw the circuit that measures in X basis?

We are now ready to run the circuits on the qasm simulator.

In [ ]:
circuits = [bellZZ,bellXX]
job = execute(circuits, backend)
result = job.result()

plot_histogram(result.get_counts(bellZZ))

As we can see, this histogram tells us that when $q_0$ is $0$, then $q_1$ is $0$ with the highest probability, and the same holds for $q_0$ and $q_1$ equal to $1$. They are maximally correlated. However, let's see what happens when we measure them in the X basis.

Again, we will find that if $q_0$ is in the $|+\rangle$ state:
    $$ |+ \rangle= \frac{(|0 \rangle+|1\rangle)} {\sqrt{2}} $$
then also $q_1$ will be $|+\rangle$, and the same will also hold for $|-\rangle$ states:
    $$ |- \rangle= \frac{(|0 \rangle-|1\rangle)} {\sqrt{2}} $$

## Mixed States
### Reduced version of the Notebook by:
Jay Gambetta, Antonio Córcoles, Andrew Cross, Anna Phan

It is important to notice that having correlated results for two qubits does not imply entanglement. 
To see this, we will prepare a classical mixed state; in other words we will measure half of the time the state $|00\rangle$ and half of the time the state $|11\rangle$, obtained applying the $X$ gate on the qubits, and compare the statistics with the Bell's State.


In [ ]:
# Create a circuit that measures the qubits in 0 and one that flips and measures them in 1:
mixed1 = QuantumCircuit(q2, c2)
mixed2 = QuantumCircuit(q2, c2)
mixed2.x(q2)
mixed1.measure(q2[0], c2[0])
mixed1.measure(q2[1], c2[1])
mixed2.measure(q2[0], c2[0])
mixed2.measure(q2[1], c2[1])

circuit_drawer(mixed1,style=qx_color_scheme())

In [ ]:
circuit_drawer(mixed2,style=qx_color_scheme())

Now we create the mixed state:

In [ ]:
mixed_state = [mixed1,mixed2]
job = execute(mixed_state, backend)
result = job.result()

counts1 = result.get_counts(mixed_state[0])
counts2 = result.get_counts(mixed_state[1])

from collections import Counter
ground = Counter(counts1)
excited = Counter(counts2)
plot_histogram(ground+excited)

At first sight, we obtain the same result as before with the Bell's State. There is no entanglement here, however. 
What happens if we look at this state in the X basis?

## Unitary Matrices
### Notebook by:
Najla Said

Now we will look at the matrices that create the Bell's State and the mixed state. To do this we should use a different simulator, not the _qasm_. Which one?

In [ ]:
backend = Aer.get_backend('') # select the appropriate backend

# Create registers
q2 = QuantumRegister(2)
c2 = ClassicalRegister(2)

# Create the Bell's State
bell = QuantumCircuit(q2, c2)
bell.h(q2[0])
bell.cx(q2[0], q2[1])

# Run the job to see the matrix that creates the entanglement
circuit = bell
job = execute(circuit, backend)
result = job.result()

print(result.get_unitary(bell, decimals=3))

Verify that the matrix of the circuit is the following:

$$ M =\frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 & 0 & 0\\ 0 & 0 & 1 & -1\\0 & 0 & 1 & 1\\1 & -1 & 0 & 0 \end{pmatrix}.$$

If we apply $M$ to the starting state $|00\rangle$, indeed, we obtain the Bell's State in that we end up in $|00\rangle$ or in $|11\rangle$ with the same probability.     
  

In [ ]:
backend = Aer.get_backend('unitary_simulator') # select the appropriate simulator

# Creazione dei registri
q2 = QuantumRegister(2)
c2 = ClassicalRegister(2)

# Creazione dello stato misto 
mix1 = QuantumCircuit(q2, c2)
mix2 = QuantumCircuit(q2, c2)
mix1.iden(q2)
mix2.x(q2)

# Eseguiamo i circuiti per ottenere le matrici
mixed_state = [mix1,mix2]
job = execute(mixed_state, backend)
result = job.result()

print(result.get_unitary(mixed_state[0], decimals=3),'\n \n',result.get_unitary(mixed_state[1],decimals=3))


Verify that the matrices of the circuits that form the mixed state are:

$$ M_1 =\begin{pmatrix} 1 & 0 & 0 & 0\\ 0 & 1 & 0 & 0\\0& 0& 1 & 0\\0 & 0 & 0 & 1 \end{pmatrix}, \hspace{2pt} M_2 =\begin{pmatrix} 0 & 0 & 0 & 1\\ 0 & 0 & 1 & 0\\0& 1& 0 & 0\\1 & 0 & 0 & 0 \end{pmatrix}.$$

If we apply $M_1$ to the starting state $|00\rangle$, we stay in $|00\rangle$ with probability 1.
On the other hand, if we apply $M_2$, we end up in $|11\rangle$, again with probability 1.

## Envariance
### Notebook by

Najla Said

Let's now demonstrate envariance, or entangled assisted invariance, following the work of this paper from Deffner arXiv:1609.07459v2 [quant-ph].

We start by generating a Bell's State, we then flip the first qubit, while the other qubit stays unchanged. We continue by flipping the second qubit, while the first stays unchanged. The envariance will bring us back to the starting state.

Indeed:
$$ \frac{1}{\sqrt{2}} (|00\rangle + |11\rangle)$$
$$ X(q_0) \rightarrow \frac{1}{\sqrt{2}} (|01\rangle + |10\rangle)$$
$$ X(q_1) \rightarrow \frac{1}{\sqrt{2}} (|11\rangle + |00\rangle)$$

In [ ]:
backend= Aer.get_backend('qasm_simulator')

q2 = QuantumRegister(2)
c2 = ClassicalRegister(2)

# Create Bell's State
env = QuantumCircuit(q2, c2)
env.h(q2[0])
env.cx(q2[0], q2[1])

# Apply the X gates
env.x(q2[0])
env.iden(q2[1])
env.iden(q2[0])
env.x(q2[1])

meas = QuantumCircuit(q2, c2)
meas.measure(q2[0], c2[0])
meas.measure(q2[1], c2[1])
env = env+meas

circuit_drawer(env,style=qx_color_scheme())

In [ ]:
circuit = env
job = execute(circuit, backend)
result = job.result()

plot_histogram(result.get_counts(env))

If we try the same experiment (apply X on a qubit, and then on the other) on two qubits $|00\rangle$ that are not entangled, we will find the classical expected result. Which one?